<a href="https://colab.research.google.com/github/larissavaladao/colab_curuai/blob/main/curuai_imgs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [1]:
#import packages used
import ee
import pandas as pd
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import math

In [2]:
#authenticate and initialize google earth engine (also necessary for geemap)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4_Fs9DMrbkRudKvta14IS3nMo0z21sx_Bx7CfQYxDPk&tc=1Ugcx1-x1y41IGqghQWdWEpEaTYcsXNuLuhUhGPckfU&cc=Lcz2Kqthom0nz_wJVdZOfDSHJmk6iWgn6tTtV9Zo-4M

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkk36PyT8OZLbeazVUt4OL8r1T_RpX7knqnPxqo3t9gIutIyFlODR4

Successfully saved authorization token.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Curuai dataset

In [4]:
#import the entire dataset and include ID column
dataset = pd.read_excel('/content/drive/MyDrive/CURUAI_PROCESS/Dataset_CFP.xlsx', sheet_name='data',na_values='NaN').loc[:383]
# dataset = pd.read_excel('Dataset_CFP.xlsx', sheet_name='data',na_values='NaN').loc[:383]


dataset['ID'] = range(len(dataset))
dataset['Local_Time'] = dataset['Local Time']
dataset['TIME_UTC'] = dataset['TIME UTC']
dataset['DEPTH_CLASS'] = dataset['DEPTH CLASS']
dataset['SAMPLE_SITE'] = dataset['SAMPLE SITE']
dataset['WATER_PERIOD'] = dataset['WATER PERIOD']
dataset['TOTAL_DEPTH'] = dataset['TOTAL DEPTH']
dataset['SAMPLING_DEPTH'] = dataset['SAMPLING DEPTH']
dataset['CHLOROPHYLL_A'] = dataset['CHLOROPHYLL A']
dataset['CHLOROPHYLL_B'] = dataset['CHLOROPHYLL B']
dataset['ODO'] = dataset['ODO ']

In [5]:
#copy the dataset and select only the variables of interest
dataset_att = dataset.copy()
dataset_att = dataset_att.loc[:,['ID','DATE', 'LATITUDE','LONGITUDE', 'MISSION']]
dataset_att.head()

,ID,DATE,LATITUDE,LONGITUDE,MISSION
0,0,2013-03-08 00:00:00,-2.25127,-55.14622,I
1,1,2013-03-08 00:00:00,-2.25127,-55.14622,I
2,2,2013-03-09 00:00:00,-2.28422,-55.22023,I
3,3,2013-03-09 00:00:00,-2.19696,-55.29953,I
4,4,2013-03-09 00:00:00,-2.221738,-55.270194,I


In [6]:
dataset_att.groupby(['MISSION'])['MISSION'].count()

MISSION
I       71
II      73
III     74
IV      36
IX      24
V       26
VI      25
VII     28
VIII    27
Name: MISSION, dtype: int64

In [7]:
#vizualize variables included in the dataset
dataset_att.columns

Index(['ID', 'DATE', 'LATITUDE', 'LONGITUDE', 'MISSION'], dtype='object')

In [8]:
#transform dataframe in a geodataframe (geometry column with point location)
gdf = gpd.GeoDataFrame(
    dataset_att, geometry=gpd.points_from_xy(dataset_att.LONGITUDE, dataset_att.LATITUDE),
    crs="EPSG:4326"
)
gdf.head()

,ID,DATE,LATITUDE,LONGITUDE,MISSION,geometry
0,0,2013-03-08 00:00:00,-2.25127,-55.14622,I,POINT (-55.14622 -2.25127)
1,1,2013-03-08 00:00:00,-2.25127,-55.14622,I,POINT (-55.14622 -2.25127)
2,2,2013-03-09 00:00:00,-2.28422,-55.22023,I,POINT (-55.22023 -2.28422)
3,3,2013-03-09 00:00:00,-2.19696,-55.29953,I,POINT (-55.29953 -2.19696)
4,4,2013-03-09 00:00:00,-2.221738,-55.270194,I,POINT (-55.27019 -2.22174)


In [9]:
#transform date and time columns in string - necessary to convert to JSON
gdf['DATE'] = gdf['DATE'].astype("str")

In [10]:
##Convert geodataframe to json - necessary to be read in GEE
dataset_json = gdf.to_json()

In [11]:
#load and select the features of the json data
data_points = json.loads(dataset_json)
data_points = data_points['features']
# data_points

In [12]:
##transform json in in gee object = feature collection
roi_points = ee.FeatureCollection(data_points)
print(roi_points.size().getInfo())

384


In [13]:
#function to insert a property with date of the point - Gee date format
def insert_date (feat):
    return feat.set('system:time_start',ee.Date.parse('YYYY-MM-dd HH:mm:ss',feat.get('DATE')))

In [14]:
#apply date function - new system:time_start property
roi_points = roi_points.map(insert_date)
print(roi_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-55.14622, -2.25127]}, 'id': '0', 'properties': {'DATE': '2013-03-08 00:00:00', 'ID': 0, 'LATITUDE': -2.25127, 'LONGITUDE': -55.14622, 'MISSION': 'I', 'system:time_start': {'type': 'Date', 'value': 1362700800000}}}


In [15]:
#create a polygon around the floodpalin area - roi
roi_poly = roi_points.geometry().buffer(500).bounds()

# Import GEE images

## Cloud Masking functions

Sentinel 2 mask

In [16]:
#define cloud masking functions
#VER NOVA MASCARA DE NUVENS DO EE
MAX_CLOUD_PROBABILITY = 20

def maskS2_1C(image):
    clouds = ee.Image(image.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return image.updateMask(isNotCloud)

'''
 * Function to mask clouds using the Sentinel-2 QA band
 * @param {ee.Image} image Sentinel-2 image
 * @return {ee.Image} cloud masked Sentinel-2 image
'''
def maskS2(image):
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0)\
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).copyProperties(image,['system:time_start'])


Landsat mask

In [17]:
#Based on [ee.Algorithms.Landsat.simpleCloudScore](https://developers.google.com/earth-engine/landsat)
#This function even calculates percentage of cloud  coverage over a particular region. Useful for screening too cloudy images
def scoreL8_TOA (image):
  #Add a cloud score band.  It is automatically called 'cloud'.
  scored = ee.Algorithms.Landsat.simpleCloudScore(image)
  #Create a mask from the cloud score and combine it with the image mask.
  mask = scored.select(['cloud']).lte(MAX_CLOUD_PROBABILITY);
  #Apply the mask to the image and display the result.
  return image.updateMask(mask)
#######################################################################
# Function to mask clouds using the quality band of Landsat 8.
def maskL8 (image):
    qa = image.select('QA_PIXEL')
  # Check that the cloud bit is off.
  # See https://www.usgs.gov/media/files/landsat-8-9-olitirs-collection-2-level-1-data-format-control-book
    mask = qa.bitwiseAnd(1 << 3).eq(0)
    return image.updateMask(mask)

## define date ranges for filtering images

In [ ]:
#define initial and final date to filter the image collection based on the
#field points date
initial_date = ee.Date(roi_points.sort('system:time_start').first().get('system:time_start'))
end_date = ee.Date(roi_points.sort('system:time_start',False).first().get('system:time_start'))

print('Data inicial de coleta: ',initial_date.format().getInfo())
print('Data final de coleta: ',end_date.format().getInfo())

Data inicial de coleta:  2013-03-08T00:00:00
Data final de coleta:  2017-09-19T00:00:00


In [18]:
def date_mission(mission,time):
  return ee.Date(roi_points.filter(ee.Filter.eq('MISSION',mission)).sort('system:time_start',time).first().get('system:time_start'))

In [224]:
mission = 'I'
advance = 17

## Import image collections and apply filters and cloud masks

###Landsat 7 collection 2 tier 1 TOA

In [225]:
#Landsat 7 collection 2 tier 1 TOA
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds
landsat7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA")\
            .filterBounds(roi_points)\
            .filterDate(date_mission(mission,True).advance(-advance, 'day'),date_mission(mission,False).advance(advance, 'day'))\
            .map(scoreL8_TOA)\
            .map(maskL8)
print(ee.Date(landsat7.first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-03-25T13:44:15
2013-04-01T13:50:26


In [ ]:
print('LD7 size',landsat7.size().getInfo())


###Landsat 8 collection 2 tier 1 TOA

In [ ]:
#Landsat 8 collection 2 tier 1 TOA
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds
landsat8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
            .filterBounds(roi_points)\
            .filterDate(date_mission(mission,True).advance(-advance, 'day'),date_mission(mission,False).advance(advance, 'day'))\
            .map(scoreL8_TOA)\
            .map(maskL8)
print(ee.Date(landsat8.first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [ ]:
print('LD8 size',landsat8.size().getInfo())


###Sentinel 2 level 1C harmonized

In [197]:
#Sentinel 2 level 1C harmonized
#filter collection by region and date (2 month prior toi the field dates and one month past) based on field points
#mask clouds

s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')\
            .filterBounds(roi_points)\
            .filterDate(date_mission(mission,True).advance(-advance, 'day'),date_mission(mission,False).advance(advance, 'day'))\

sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
            .filterBounds(roi_points)\
            .filterDate(date_mission(mission,True).advance(-advance, 'day'),date_mission(mission,False).advance(advance, 'day'))\


s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(primary=sentinel2,
  secondary=s2Clouds,condition=ee.Filter.equals(leftField='system:index', rightField='system:index'))

In [ ]:
sentinel2 = ee.ImageCollection(s2SrWithCloudMask).map(maskS2_1C).map(maskS2)

print(ee.Date(sentinel2.first().get('system:time_start')).format().getInfo())
print(ee.Date(sentinel2.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

In [ ]:
print('sentinel 2 size',sentinel2.size().getInfo())

### visualização das imagens

comparação LD8/S2 - mosaico de todo periodo da missão

In [143]:
landsat8_med = landsat8.median().clip(roi_poly)
sentinel2_med = sentinel2.median().clip(roi_poly)

left_layer = geemap.ee_tile_layer(landsat8_med, {'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'landsat8_med')
right_layer = geemap.ee_tile_layer(sentinel2_med,{'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'sentinel2_med')

In [ ]:
Map = geemap.Map()
Map.centerObject(roi_points,10)
Map.split_map(left_layer, right_layer)
Map

Primeira imagem com menos nuvens da colação de iamgens disponiveis durante a missão

In [ ]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
Map.addLayer(landsat7.sort('CLOUD_COVER').first(), {'bands':['B3','B2','B1'],'min':0,'max':0.3}, 'landsat7')
Map.addLayer(landsat8.sort('CLOUD_COVER').first(), {'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'landsat8')
Map.addLayer(sentinel2.sort('CLOUDY_PIXEL_PERCENTAGE').first(), {'bands':['B4','B3','B2'],'min':0,'max':0.3}, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

# Preprocessing

## Padronize band names and projection

In [ ]:
## unify bands spatial resolution to 30m (mean reducer) and projection
#reprojetar para mesmo epsg opção trocando resolução espacial e fazendo resample
#agregar pixels - mesma projeção resolução espacial menor
# def reproject_reduceResolution(img):
#     return img.reduceResolution(reducer=ee.Reducer.mean(),
#       maxPixels=1024
#     ).reproject(crs=prj.crs(),
#                 scale=prj.nominalScale()).copyProperties(img,['system:time_start'])

#reamostrar pixels - muda projeção e resolução espacial diferente (pode ser maior)
# def reproject_resample(img):
#   return img.setDefaultProjection(crs=prj.crs(), scale=prj.nominalScale()).resample('bicubic').reproject(crs=prj.crs(),
#   scale=prj.nominalScale()).copyProperties(img,['system:time_start'])


# #apenas com reprojecao sem mudar escala nem reamostrar
# def reprojectImg(img):
#   return img.reproject(crs=prj.crs(),scale=img.select('green').projection().nominalScale()).copyProperties(img,['system:time_start'])

In [226]:
#renomear bandas
prj = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_228061_20161111').select(['B4']).projection()

name_bands = ['blue','green','red','nir','swir']


ld7 = landsat7.map(lambda img: img.select(['B1','B2','B3','B4','B5'])\
                        .rename(name_bands)
                        .setDefaultProjection(prj))

In [227]:
print(ld7.first().select('red').projection().getInfo())

{'type': 'Projection', 'crs': 'EPSG:32621', 'transform': [30, 0, 540285, 0, -30, -44085]}


In [ ]:
ld8 = landsat8.map(lambda img: img.select(['B2','B3','B4','B5','B6'])\
                        .rename(name_bands)
                        .setDefaultProjection(prj))

In [ ]:
print(ld8.first().select('red').projection().getInfo())

In [ ]:
s2 = sentinel2.map(lambda img: img.select(['B2','B3','B4','B8','B11'])\
                        .rename(name_bands)
                        .setDefaultProjection(prj))

In [ ]:
print(s2.first().select('red').projection().getInfo())

## ATM correction

Dark Object Subtraction

In [169]:
def darkObjectSubtraction(img):
    imgReducedVal = (img.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': img.geometry(),
      'crs': img.select('red').projection().crs(),
      'scale': img.select('red').projection().nominalScale(),
      'maxPixels': 1e9}))

    offset = imgReducedVal.toImage(img.bandNames())

    imgAdj = img.subtract(offset)

    return imgAdj.set('system:time_start',img.get('system:time_start'))

In [228]:
## aplicar correção atm
##DOS
ld7_cor = ld7.map(darkObjectSubtraction)

In [ ]:
ld8_cor = ld8.map(darkObjectSubtraction)

In [170]:
s2_cor = s2.map(darkObjectSubtraction)

In [ ]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
vis =  {'bands':['red','green','blue'],'min':0,'max':0.04}
Map.addLayer(ld7_cor.first(),vis, 'landsat7')
Map.addLayer(ld8_cor.first(), vis, 'landsat8')
Map.addLayer(s2_cor.first(), vis, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

## Transform into remote sensing  reflectance and sunglint correction

In [172]:
#dividir imagem corrigida por pi
# Rrs_sat_ac = Rsat_ac / pi

#fazer deglint
#Rrs_sat_ac_deglint(VNIR) = Rrs_sat_ac (VNIR) − Rrs_sat_ac (SWIR)
#correção no artigo INPE CURUAI
def deglint (img):
    Rrs = img.divide(math.pi)
    deglint = Rrs.select(['blue','green','red','nir'])\
    .subtract(Rrs.select('swir'))

    return (deglint.updateMask(deglint.select(['red']).gt(0))
    .copyProperties(img,['system:time_start']))

In [229]:
landsat7_rs = ld7_cor.map(deglint)

In [ ]:
landsat8_rs = ld8_cor.map(deglint)

In [ ]:
sentinel2_rs = s2_cor.map(deglint)

In [ ]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(roi_points,10)
vis =  {'bands':['red','green','blue'],'min':0.01,'max':0.06}
Map.addLayer(landsat7_rs.first(),vis, 'landsat7')
Map.addLayer(landsat8_rs.first(), vis, 'landsat8')
Map.addLayer(sentinel2_rs.first(), vis, 'sentinel2')

Map.addLayer(roi_points, {'color':'darkred'}, 'Data Points');
# Map.addLayer(roi_poly, {'color':'darkred'}, 'Data polygon');
Map

# Sample data points pixel values

filter images that fall within a 16-day window period from each field point date

In [150]:
def imgs_points(collection):
    def wrap(feat):
        date_point = ee.Date(feat.get('system:time_start'))
        data1 = date_point.advance(-16,'day')\
        .format('yyyy-MM-dd')

        data2 = date_point.advance(16,'day')\
        .format('yyyy-MM-dd')

        filtro = collection.filterDate(data1,data2)\
        .filterBounds(feat.geometry())\
        .map(lambda img: img.set('dif_date_point',ee.Date(img.get('system:time_start')).difference(date_point, 'day'))\
            .copyProperties(feat,['ID']))


        return ee.ImageCollection(filtro.limit(4)) ####sera q precisa converter??

    return wrap

obtain statistics for the same location of the field point - with a 3 pixel window - and filter out if more than 4 pixels are masked   

In [152]:
def sample_point(img):
  feat = roi_points.filter(ee.Filter.eq('ID',img.get('ID'))).first()
  geom = feat.geometry().buffer(45).bounds()

  mean = img.reduceRegion(geometry=geom,
                          scale=prj.nominalScale(),
                          crs=prj.crs(),
                          reducer=ee.Reducer.mean())
  median = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.median())
  minMax = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.minMax())
  count = img.reduceRegion(geometry=geom,
                            scale=prj.nominalScale(),
                            crs=prj.crs(),
                            reducer=ee.Reducer.count())
  return feat.set({
      'dif_date_point': img.get('dif_date_point'),
      'blue_mean':mean.get('blue'),
      'green_mean':mean.get('green'),
      'red_mean':mean.get('red'),
      'nir_mean':mean.get('nir'),
      'blue_median':median.get('blue'),
      'green_meadin':median.get('green'),
      'red_median':median.get('red'),
      'nir_median':median.get('nir'),
      'blue_min':minMax.get('blue_min'),
      'green_min':minMax.get('green_min'),
      'red_min':minMax.get('red_min'),
      'nir_min':minMax.get('nir_min'),
      'blue_max':minMax.get('blue_max'),
      'green_max':minMax.get('green_max'),
      'red_max':minMax.get('red_max'),
      'nir_max':minMax.get('nir_max'),
      "count_pixel":count.get('red')
      })
  return minMax
filter_count = ee.Filter.gt('count_pixel',4)

### Landsat 7

In [230]:
img_pointsLD7 = ee.ImageCollection(roi_points.map(imgs_points(landsat7_rs)).flatten().toList(2000))

In [ ]:
img_pointsLD7.size()


In [ ]:
img_pointsLD7.aggregate_count_distinct('ID')

In [ ]:
img_pointsLD7.limit(5)

In [231]:
reduced_LD7 = ee.FeatureCollection(img_pointsLD7.map(sample_point)).filter(filter_count)

In [120]:
reduced_LD7.limit(2)

In [121]:
reduced_LD7.size().getInfo()

66

In [122]:
reduced_LD7.aggregate_count_distinct('ID').getInfo()

26

## Landsat 8

In [ ]:
img_pointsLD8 = ee.ImageCollection(roi_points.map(imgs_points(landsat8_rs)).flatten().toList(2000))

In [189]:
img_pointsLD8.size()


In [ ]:
img_pointsLD8.aggregate_count_distinct('ID')

In [ ]:
img_pointsLD8.limit(5)

In [ ]:
reduced_LD8 = ee.FeatureCollection(img_pointsLD8.map(sample_point)).filter(filter_count)

In [125]:
reduced_LD8.size()


In [126]:
reduced_LD8.aggregate_count_distinct('ID')

In [127]:
reduced_LD8.limit(2)

##Sentinel 2

In [156]:
img_pointsS2 = ee.ImageCollection(roi_points.map(imgs_points(sentinel2_rs)).flatten().toList(2000))

In [ ]:
img_pointsS2.size()

In [ ]:
img_pointsS2.aggregate_count_distinct('ID')

In [157]:
reduced_S2 = ee.FeatureCollection(img_pointsS2.map(sample_point)).filter(filter_count)

In [158]:
reduced_S2.size()


In [131]:
reduced_S2.aggregate_count_distinct('ID')

In [132]:
reduced_S2.limit(2)

## Visualize sampled image by defining field point ID

In [ ]:
Map = geemap.Map()
Map.centerObject(roi_points,10)
id = 360
vis_params={'bands':['red','green','blue'],'min':0,'max':0.05}
Map.addLayer(img_pointsLD7.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat7')
Map.addLayer(img_pointsLD8.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'landsat8')
Map.addLayer(img_pointsS2.filter(ee.Filter.eq('ID',id)).sort('dif_date_point').first(), vis_params, 'sentinel2')

Map.addLayer(roi_points.filter(ee.Filter.eq('ID',id)).geometry(), {'color':'darkred'}, 'Data Points');
Map

# Export data as CSV file

In [232]:
import geemap
#Exportar tabelas
#LD7
geemap.ee_to_csv(reduced_LD7, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD7_DOS_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/I_LD7_DOS_data.csv


In [220]:
geemap.ee_to_csv(reduced_LD8, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_LD8_DOS_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/II_LD8_DOS_data.csv


In [161]:
geemap.ee_to_csv(reduced_S2, '/content/drive/MyDrive/CURUAI_PROCESS/'+mission+'_S2_DOS_data.csv', selectors=None, verbose=True, timeout=300, proxies=None)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/CURUAI_PROCESS/VI_S2_DOS_data.csv


CORREÇÂO ATM:


1.   ACOLITE https://github.com/acolite/acolite
2.   6S com imagem da Dinha e parametros do artigo... talvez seja necessaria mais uma correção para isso



